## Setup Task Environment

Different applications come with different requirements for the runtime environment. This section describes how the shell environment for a task can be configured.

The task environment is defined via a Python dictionary, as part of the task description:

```python
cud = rp.TaskDescription()

cud.executable  = '/bin/echo'
cud.arguments   = ['$RP_TASK_ID greets $TEST']
cud.environment = {'TEST' : 'jabberwocky'}
```

<b>Warning:</b> Always call <b>radical.pilot.Session.close()</b> before your application terminates to terminate all lingering pilots. You can use the function argument <b>cleanup=True</b> to delete the entries of the session from the database. If you need to retain those data, use the function argument <b>download=True.</b>

This makes the environment variable <i>TEST</i> available during task execution. Some other variables, such as the <i>RP_TASK_ID</i> above, are set by RP internally and are here used for demonstration.

### Running the Example

Below is an example that uses the code above to run a bag of echo commands.

As usual, we start by importing the <i> radical.pilot</i> module and initializing the reporter facility used for printing well formatted runtime and progress information.

In [ ]:
import os
import sys

verbose  = os.environ.get('RADICAL_PILOT_VERBOSE', 'REPORT')
os.environ['RADICAL_PILOT_VERBOSE'] = verbose

import radical.pilot as rp
import radical.utils as ru

report = ru.Reporter(name='radical.pilot')
report.title('Getting Started (RP version %s)' % rp.version)

We will now import the dotenv module for fetching our environment variables. To create a new Session, you need to provide the URL of a MongoDB server which we will fetch from our .env file.

We will set the resource value to <i>'local.localhost'</i>. Using a resource key other than local.localhost implicitly tells RADICAL-Pilot that it is targeting a remote resource.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

RADICAL_PILOT_DBURL = os.getenv("RADICAL_PILOT_DBURL")
os.environ['RADICAL_PILOT_DBURL'] = RADICAL_PILOT_DBURL
resource = 'local.localhost'
session = rp.Session()

All other pilot code is now tried/excepted. If an exception is caught, we can rely on the session object to exist and be valid, and we can thus tear the whole RP stack down via a <i>'session.close()'</i> call in the <i>'finally'</i> clause.

In [ ]:
def create_pilot_descriptions(resource):
    # read the config used for resource details
    report.info('read config')
    config = ru.read_json('../config.json')
    report.ok('>>ok\n')

    report.header('submit pilots')

    # Define an [n]-core local pilot that runs for [x] minutes
    # Here we use a dict to initialize the description object
    pd_init = {
               'resource'      : resource,
               'runtime'       : 15,  # pilot runtime (min)
               'exit_on_error' : True,
               'project'       : config[resource].get('project', None),
               'queue'         : config[resource].get('queue', None),
               'access_schema' : config[resource].get('schema', None),
               'cores'         : config[resource].get('cores', 1),
               'gpus'          : config[resource].get('gpus', 0),
              }
    pdesc = rp.PilotDescription(pd_init)
    return pdesc

In [ ]:
def launch_pilots(session,pdesc):
    # Add a Pilot Manager. Pilot managers manage one or more Pilots.
    pmgr = rp.PilotManager(session=session)
    # Launch the pilot.
    pilot = pmgr.submit_pilots(pdesc)
    return pilot


In [ ]:
def submit_tasks(pilot):
    report.header('submit tasks')

    # Register the Pilot in a TaskManager object.
    tmgr = rp.TaskManager(session=session)
    tmgr.add_pilots(pilot)

    # Create a workload of Tasks.
    # Each task runs a specific `echo` command

    n = 10   # number of tasks to run
    report.info('create %d task description(s)\n\t' % n)

    tds = list()
    for i in range(0, n):

        # create a new Task description, and fill it.
        # Here we don't use dict initialization.
        td = rp.TaskDescription()
        td.environment = {'TEST' : 'jabberwocky'}
        td.executable  = '/bin/echo'
        td.arguments   = ['$RP_TASK_ID greets $TEST']

        tds.append(td)
        report.progress()
    report.ok('>>ok\n')

    # Submit the previously created Task descriptions to the
    # PilotManager. This will trigger the selected scheduler to start
    # assigning Tasks to the Pilots.
    tasks = tmgr.submit_tasks(tds)

    # Wait for all tasks to reach a final state (DONE, CANCELED or FAILED).
    report.header('gather results')
    tmgr.wait_tasks()
    return tasks

In [ ]:
def report_task_progress(tasks):
    report.info('\n')
    for task in tasks:
        report.plain('  * %s: %s, exit: %3s, out: %s\n'
                % (task.uid, task.state[:4],
                    task.exit_code, task.stdout.strip()[:35]))

In [ ]:
try:
    pdesc = create_pilot_descriptions('local.localhost')
    pilots = launch_pilots(session,pdesc)
    tasks = submit_tasks(pilots)
    report_task_progress(tasks)

except Exception as e:
    # Something unexpected happened in the pilot code above
    report.error('caught Exception: %s\n' % e)
    raise

except (KeyboardInterrupt, SystemExit):
    # the callback called sys.exit(), and we can here catch the
    # corresponding KeyboardInterrupt exception for shutdown.  We also catch
    # SystemExit (which gets raised if the main threads exits for some other
    # reason).
    report.warn('exit requested\n')

finally:
    # always clean up the session, no matter if we caught an exception or
    # not.  This will kill all remaining pilots.
    report.header('finalize')
    session.close(cleanup=True)

report.header()